- 잠재공간을 이용하는 두가지  생성 모델
  - 변이형 오토인코더(VAE)
  - 생성적 적대 신경망(GAN)

- 자기회귀모델(autoregressive model)
  - 잠재 확률변수가 아니라 시퀀스에 있는 이전값을 바탕으로 예측을 만듦
  

In [1]:
# 자기회귀모델 LSTM(텍스트 데이터),  PixelCNN(이미지 데이터)

In [2]:
# https://drive.google.com/file/d/1CPCu895GkGPrvChspl7iPBkoqpBzKCOO/view?usp=sharing

In [3]:
!unzip '/content/drive/MyDrive/레시피데이터셋/Epicurious - Recipes with Rating and Nutrition.zip'

Archive:  /content/drive/MyDrive/레시피데이터셋/Epicurious - Recipes with Rating and Nutrition.zip
  inflating: epi_r.csv               
  inflating: full_format_recipes.json  
  inflating: recipe.py               
  inflating: utils.py                


In [4]:
import pandas as pd
import numpy as np
import json

In [5]:
with open('/content/full_format_recipes.json')as json_data:
  recipe_data =  json.load(json_data)
recipe_data[0].keys()


dict_keys(['directions', 'fat', 'date', 'categories', 'calories', 'desc', 'protein', 'rating', 'title', 'ingredients', 'sodium'])

In [6]:
recipe_data[0]['title'],recipe_data[0]['directions']

('Lentil, Apple, and Turkey Wrap ',
 ['1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool.',
  '2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper.',
  '3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.'])

In [7]:
temp = recipe_data[0]
temp['title']+" : " + ' '.join(temp['directions'])

'Lentil, Apple, and Turkey Wrap  : 1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool. 2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper. 3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.'

In [8]:
filtered_data = []
for rd in recipe_data:
  if ('title' in rd) and (rd['title'] is not None) and (rd['directions'] is not None):
    filtered_data.append( rd['title']+" : " + ' '.join(rd['directions']) )

In [9]:
filtered_data[:3]

['Lentil, Apple, and Turkey Wrap  : 1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool. 2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper. 3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.',
 'Boudin Blanc Terrine with Red Onion Confit  : Combine first 9 ingredients in heavy medium saucepan. Add 3 shallo

- 텍스트를 토큰화
  - 단어나 문자로 나누는 작업
- 단어 토큰
  - 모든 텍스트를 소문자로 변환
  - 어휘사전(vocabulary)이 매우 클 수도 있음
  - 어간(stem) : 현재 과거, 진행
  - 마침표와 쉼표를 제거
  - 훈련어휘에 없는 단어는 예측 불가
- 문자 토큰
  - 시퀀스를 생성해서 훈련 어휘에 없는 새로운 단어를 생성가능
  - 어휘사전이 매우 작아진다 그래서 마지막 출력층에 학습할 가중치가 적기때문에 훈련속도에 유리

In [10]:
# 텍스트 전처리
import string
import re
def pad_punctuation(s):
  s = re.sub(f"([{string.punctuation}])", r' \1', s)
  s = re.sub(' +',' ',s)
  return s
pad_punctuation("!!!abc#$")

' ! ! !abc # $'

In [11]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
text_data = [pad_punctuation(x) for x in filtered_data]
text_data[:5]

['Lentil , Apple , and Turkey Wrap : 1 . Place the stock , lentils , celery , carrot , thyme , and salt in a medium saucepan and bring to a boil . Reduce heat to low and simmer until the lentils are tender , about 30 minutes , depending on the lentils . (If they begin to dry out , add water as needed . ) Remove and discard the thyme . Drain and transfer the mixture to a bowl ; let cool . 2 . Fold in the tomato , apple , lemon juice , and olive oil . Season with the pepper . 3 . To assemble a wrap , place 1 lavash sheet on a clean work surface . Spread some of the lentil mixture on the end nearest you , leaving a 1 -inch border . Top with several slices of turkey , then some of the lettuce . Roll up the lavash , slice crosswise , and serve . If using tortillas , spread the lentils in the center , top with the turkey and lettuce , and fold up the bottom , left side , and right side before rolling away from you .',
 'Boudin Blanc Terrine with Red Onion Confit : Combine first 9 ingredients

In [13]:
# 텐서플로의 데이터셋으로 변환
import tensorflow as  tf

In [14]:
text_ds = tf.data.Dataset.from_tensor_slices(text_data).batch(32).shuffle(1000)

In [15]:
import tensorflow.keras.layers as layers

In [16]:
vectorize_layer =  layers.TextVectorization(
    standardize= 'lower',  # 소문자로 변경
    max_tokens = 10000, # 자주 등장하는 1000개의 단어에 정수를 부여
    output_mode = 'int',
    output_sequence_length = 200+1  # 시퀀스의 길이가 201개의 토큰으로 자르거나 패팅
)

In [17]:
#벡터화를 한 층
vectorize_layer.adapt(text_ds)  # TextVectorization층을 훈련데이터에 적용
vocab =  vectorize_layer.get_vocabulary() # vocab변수에 단어리스트를 저장
vocab

['',
 '[UNK]',
 '.',
 ',',
 'and',
 'to',
 'in',
 'the',
 'with',
 'a',
 'until',
 '1',
 'minutes',
 'of',
 'heat',
 'add',
 'about',
 'over',
 'bowl',
 ';',
 ':',
 '2',
 'salt',
 'into',
 'on',
 'large',
 'medium',
 'mixture',
 ')',
 'pepper',
 'oil',
 'is',
 'water',
 'transfer',
 '3',
 'stir',
 'or',
 'cook',
 'pan',
 'remaining',
 'then',
 'oven',
 'for',
 'stirring',
 'cover',
 'from',
 'butter',
 'skillet',
 'sugar',
 'at',
 'sauce',
 'cup',
 '/2',
 'baking',
 '5',
 'cool',
 '-inch',
 'season',
 'be',
 'place',
 'it',
 'small',
 '4',
 'each',
 'let',
 'serve',
 'boil',
 'simmer',
 'remove',
 'top',
 'whisk',
 '/4',
 'cut',
 'heavy',
 'ahead',
 'are',
 'saucepan',
 'cream',
 'garlic',
 'bring',
 'chicken',
 '10',
 'preheat',
 'using',
 'sprinkle',
 'tablespoons',
 'occasionally',
 'just',
 'brown',
 'tender',
 'dough',
 'bake',
 'side',
 'spoon',
 'golden',
 'up',
 'pot',
 'through',
 'combine',
 'juice',
 'sheet',
 'if',
 'hours',
 'pour',
 'chill',
 'lemon',
 'together',
 'flour

In [18]:
import numpy as np
import json
import re
import string
import tensorflow as tf
from tensorflow.keras import layers,models,callbacks,losses

In [19]:
VOCAB_SIZE = 10000
MAX_LEN = 200
EMBEDDING_DIM = 100
N_UNITS = 128
VALIDATION_SPLIT = 0.2
SEED = 42
BATCH_SIZE = 32
EPOCHS = 25

In [20]:
# 토큰,단어 매핑
for i, word in enumerate( vocab[:10]):
  print(f"{i}:{word}")

0:
1:[UNK]
2:.
3:,
4:and
5:to
6:in
7:the
8:with
9:a


In [21]:
example_data = text_data[0]
print(example_data)
example_tokenised =  vectorize_layer(example_data)
print(example_tokenised.numpy())

Lentil , Apple , and Turkey Wrap : 1 . Place the stock , lentils , celery , carrot , thyme , and salt in a medium saucepan and bring to a boil . Reduce heat to low and simmer until the lentils are tender , about 30 minutes , depending on the lentils . (If they begin to dry out , add water as needed . ) Remove and discard the thyme . Drain and transfer the mixture to a bowl ; let cool . 2 . Fold in the tomato , apple , lemon juice , and olive oil . Season with the pepper . 3 . To assemble a wrap , place 1 lavash sheet on a clean work surface . Spread some of the lentil mixture on the end nearest you , leaving a 1 -inch border . Top with several slices of turkey , then some of the lettuce . Roll up the lavash , slice crosswise , and serve . If using tortillas , spread the lentils in the center , top with the turkey and lettuce , and fold up the bottom , left side , and right side before rolling away from you .
[1841    3  431    3    4  218  206   20   11    2   59    7  298    3
  924  

In [22]:
# 훈련세트 만들기
# LSTM은 단어시퀀스가 주어지면 이 시퀀스의 다음 단어를 예측하도록 훈련
# 전체시퀀스를 한 토큰 이동시켜서 타깃 변수를 만듦
def prepare_inputs(text):
  text =  tf.expand_dims(text,-1)
  tokenized_sentences =  vectorize_layer(text)
  x = tokenized_sentences[:, :-1]
  y =  tokenized_sentences[:, 1:]
  return x,y
train_ds = text_ds.map(prepare_inputs)

In [23]:
# LSTM : inputlayer - embedding - lstm - dense

In [24]:
inputs = layers.Input(shape=(None,),dtype = 'int32') # 시퀀스의 길이는 가변적
x = layers.Embedding(VOCAB_SIZE,EMBEDDING_DIM)(inputs)  # 어휘사전의크기, 임베딩 벡터의 차원
x = layers.LSTM(N_UNITS,return_sequences=True)(x) # 은닉벡터의 차원을 지정, 전체 타임스템프의 은닉상태 반환
outputs = layers.Dense(VOCAB_SIZE,activation = 'softmax')(x) # 확률 벡터로 변환
lstm = models.Model(inputs,outputs)
lstm.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         1000000   
                                                                 
 lstm (LSTM)                 (None, None, 128)         117248    
                                                                 
 dense (Dense)               (None, None, 10000)       1290000   
                                                                 
Total params: 2407248 (9.18 MB)
Trainable params: 2407248 (9.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
# 콜벡함수
model_checkpoint_callback = callbacks.ModelCheckpoint(
    'checkpoint.ckpt',
    save_weights_only = True,
    save_freq = 'epoch',
    verbose = 0
)
tensorboard_callback = callbacks.TensorBoard(log_dir = './logs')

In [28]:
# TextGenerator 체크포인트 만들기
class TextGenerator(callbacks.Callback):
  # 어휘사전(단어에서 토큰)
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature): # temperature 사용해서 확률을 업데이트
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
      # 생성과정을 시작하기 위해서 모델에 제공하는 단어의 문자열
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0: # 조건에 부합될때까지 시퀀스를 생성
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature) # temperature기반으로 다음단어 예측
            info.append({"prompt": start_prompt, "word_probs": probs})
            start_tokens.append(sample_token)  # 다음반복을 위해서 새로운 단어를 추가
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\n생성된 텍스트:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("recipe for", max_tokens=100, temperature=1.0)

text_generator = TextGenerator(vocab)

In [29]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
lstm.compile('adam',loss_fn)
lstm.fit(train_ds,epochs=25,
         callbacks=[model_checkpoint_callback,tensorboard_callback,text_generator])

Epoch 1/25
629/629 [==============================] - ETA: 0s - loss: 2.1201
생성된 텍스트:
recipe for 450°f primavera : add fresh herbs , mozzarella , quartered , curry , onion , rice , and salt ; stir to paste . whisk until yolks are smooth and smooth . using tongs , transfer to a buttered /2 cup of rice , reserving 1 sides and roast until flavors to prevent cabbage forms . 

629/629 [==============================] - 51s 77ms/step - loss: 2.1201
Epoch 2/25
628/629 [============================>.] - ETA: 0s - loss: 2.0071
생성된 텍스트:
recipe for already salmon . *available at charcoal thermometer consistency of curd registers eastern is burn ; plunging mussels and bake for 40 hen with turning occasionally . transfer tomatoes to puddings and cut tomatoes : place garlic free of shrimp strips in burner . pour into large saucepan with ice water . drain . meanwhile , add lentils , and sugar in a large bowl and boil until mixture is just thermometer registers dente . using oiled wet back of a large 